In [263]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from Ancla import Featurizer, RTDataset, BuModel, LandscapeExplorer, BottomUpResNet
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [264]:
training_data = pd.read_csv(r"D:\DB_FullSequencesDistinct_noJurkat.csv")

C:\Users\elabo\AppData\Local\Temp\ipykernel_24156\966019312.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  training_data = pd.read_csv(r"D:\DB_FullSequencesDistinct_noJurkat.csv")


In [265]:
training_data = training_data[training_data['FullSequence'].str.contains('Phospho') | training_data['FullSequence'].str.contains('Carbamido') | training_data['FullSequence'].str.contains('Acetyl')]

#base sequence less than 50 amino acids
training_data = training_data[training_data['BaseSequence'].str.len() <= 50]


In [266]:
vocab = pd.read_csv(r"D:\OtherPeptideResultsForTraining\vocab.csv")

# make vocab a dictionary
vocab = dict(zip(vocab["Id"], vocab["Token"]))
#swap keys and values
vocab = {v: k for k, v in vocab.items()}


In [267]:
# change the word Phosphorylation to Phospho
training_data = training_data.replace('Phosphorylation', 'Phospho', regex=True)

X, y = Featurizer.featurize_all_full_sequences(training_data, vocab, 55)

Featurizing: 100%|██████████| 58833/58833 [00:00<00:00, 1584610.61it/s]


In [268]:
X = Featurizer.pca_features(X)[:][0]
y = Featurizer.normalize_targets(y)

In [271]:
# sequential model to be used to predict retention time

model = nn.Sequential(
    nn.Linear(55, 25),
    nn.ReLU(),
    nn.Linear(25, 1)
)
model.double()

# Divide training features into train and test
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
X_validate, X_test, y_validate, y_test = train_test_split(X_validate, y_validate, test_size=0.5, shuffle=True, random_state=42)

# Create data sets
train_dataset = RTDataset(X_train, y_train)
validation_dataset = RTDataset(X_validate, y_validate)
test_dataset = RTDataset(X_test, y_test)

# Create data loaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

criterion = torch.nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov=True)

# Explore the landscape 
explorer = LandscapeExplorer(model = model, criterion = criterion, optimizer = optimizer,
                                training_dataloader = train_loader,
                                validation_dataloader = validation_loader,
                                testing_dataset = test_dataset,
                                num_points = 10, range_ = 1)

explorer.train(epochs = 500)
explorer.test()

Epoch :   1%|          | 1/100 [00:03<05:34,  3.38s/it]

Training Loss : 0.054001757793561145, Validation Loss : 0.048025247886015116


Epoch :   2%|▏         | 2/100 [00:06<05:26,  3.33s/it]

Training Loss : 0.04743888768376664, Validation Loss : 0.04751614440160235


Epoch :   3%|▎         | 3/100 [00:10<05:23,  3.33s/it]

...

Training Loss : 0.04108537025051645, Validation Loss : 0.04093497230563764


Epoch :  48%|████▊     | 48/100 [02:36<02:51,  3.31s/it]

Training Loss : 0.04103182998327113, Validation Loss : 0.040835822575014086


Epoch :  48%|████▊     | 48/100 [02:37<02:50,  3.28s/it]


KeyboardInterrupt: 

In [ ]:
train_dataset